<a href="https://colab.research.google.com/github/Reigns-Liu/Colab-notbooks/blob/main/church_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install lets-plot

In [ ]:
pip install plotnine

In [ ]:
import pandas as pa
import polars as pl
import plotly.express as px
import lets_plot as lp
from lets_plot import *
LetsPlot.setup_html()

In [ ]:
patterns = pl.read_parquet('/content/patterns.parquet')
places = pl.read_parquet('/content/places.parquet')

In [ ]:
places.select('placekey', 'location_name').with_columns([
    pl.col('location_name').str.len_chars().alias('count')
])

placekey,location_name,count
str,str,u32
"""zzy-223@5qc-x49-gp9""","""The Church of Jesus Christ of …",47
"""223-222@8g7-46m-pvz""","""Silver Leaf Baptist Church""",26
"""223-222@8g7-33w-xqz""","""Church of Jacksonville""",22
"""zzw-222@8g6-ykb-v4v""","""Alliance Of Divine Love Inc""",27
"""223-222@8g7-2gt-7dv""","""Truth Chapel""",12
…,…,…
"""222-222@8g7-n34-k9f""","""Feed My Sheep Ltd""",17
"""zzw-222@8g8-3hy-gp9""","""Good Hope Family Worship Churc…",31
"""224-222@8g9-zyy-fvf""","""Parkway Baptist Temple""",22


In [ ]:
ggplot(patterns.limit(100), aes(x='raw_visit_counts')) + ggsize(700, 300) + \
    geom_density(color='dark_green', alpha=.7) + scale_fill_brewer(type='seq') + \
    theme(panel_grid_major_x='blank')

In [ ]:
places.select('placekey', 'location_name')

placekey,location_name
str,str
"""zzy-223@5qc-x49-gp9""","""The Church of Jesus Christ of …"
"""223-222@8g7-46m-pvz""","""Silver Leaf Baptist Church"""
"""223-222@8g7-33w-xqz""","""Church of Jacksonville"""
"""zzw-222@8g6-ykb-v4v""","""Alliance Of Divine Love Inc"""
"""223-222@8g7-2gt-7dv""","""Truth Chapel"""
…,…
"""222-222@8g7-n34-k9f""","""Feed My Sheep Ltd"""
"""zzw-222@8g8-3hy-gp9""","""Good Hope Family Worship Churc…"
"""224-222@8g9-zyy-fvf""","""Parkway Baptist Temple"""


In [ ]:
mergedata = places.join(patterns, on="placekey", how="left")

##What differences are there between iPhone and Android users when comparing visits to The Church of Jesus Christ buildings of Latter-day Saints in Utah and Georgia?

In [ ]:
GA = mergedata.select('placekey', 'region','location_name','city').filter(pl.col('region') == 'GA',pl.col('location_name').str.contains('Church of Jesus Christ of'))
UT = mergedata.select('placekey', 'region','location_name','city').filter(pl.col('region') == 'UT',pl.col('location_name').str.contains('Church of Jesus Christ of'))


GA_data = pl.concat([GA])
UT_data = pl.concat([UT])

GA_1 = patterns.join(GA_data, on='placekey', how='inner')
UT_1 = patterns.join(UT_data, on='placekey', how='inner')


GA_device = GA_1.select(['placekey',"region", "device_type","location_name"]),(pl.count().alias("visit_count"),pl.col('location_name').str.contains('Church of Jesus Christ of'))
UT_device = UT_1.select(['placekey',"region", "device_type","location_name"]),(pl.count().alias("visit_count"),pl.col('location_name').str.contains('Church of Jesus Christ of'))


merge_data = pl.concat([GA_1, UT_1])

device = merge_data.select("placekey", "device_type", "region",).explode("device_type").unnest("device_type")

visits = merge_data.select("placekey", "raw_visit_counts", "location_name",  "related_same_day_brand", "bucketed_dwell_times", "popularity_by_day", "popularity_by_hour")

data = visits.unique(subset="placekey").join(device, on="placekey", how="inner")

Q1 = data.group_by("region", "key").agg(pl.sum("value").alias("LDS Devices"))
Q1

<ipython-input-12-805494ed4ab6>:12: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  GA_device = GA_1.select(['placekey',"region", "device_type","location_name"]),(pl.count().alias("visit_count"),pl.col('location_name').str.contains('Church of Jesus Christ of'))
<ipython-input-12-805494ed4ab6>:13: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  UT_device = UT_1.select(['placekey',"region", "device_type","location_name"]),(pl.count().alias("visit_count"),pl.col('location_name').str.contains('Church of Jesus Christ of'))


region,key,LDS Devices
str,str,i32
"""UT""","""ios""",566618
"""UT""","""android""",733773
"""GA""","""ios""",19416
"""GA""","""android""",21480


In [ ]:
ggplot(Q1) + aes(x='region', y='LDS Devices', fill='key') +\
    geom_bar(stat='identity', position='dodge') +\
    ggtitle('Bar Chart Separated by Region') +\
        xlab('Region') +\
        ylab('Value')

The most noticeable aspect of these graphs is the significant disparity in figures between Georgia and Utah. Given that Utah has a larger LDS population, it's expected that the number of devices logged at locations in Utah would be considerably higher.

##Compare hourly usage patterns between The Church of Jesus Christ of Latter-day Saints and the other churches in each state.

In [ ]:
hourly = mergedata.select("placekey", "region", "location_name", "popularity_by_hour").explode("popularity_by_hour")
hourly = hourly.with_columns(pl.col("popularity_by_hour").cum_count().over("placekey").alias("hour"))

hourly = hourly.select(["placekey", "region", "location_name", "popularity_by_hour", "hour"])

max_popularity = pl.col("popularity_by_hour").max().over("placekey")
hour_most_popular = pl.when(pl.col("popularity_by_hour") == max_popularity)\
    .then(pl.col("hour")).alias("hour_most_popular")

hourly = hourly.with_columns([
    max_popularity.alias("max_popularity"),
    hour_most_popular
])


hour_most_popular = hourly.filter(pl.col("hour_most_popular").is_not_null())


hour_most_popular_church = hour_most_popular.group_by("location_name", "region").agg([
    pl.col("hour_most_popular").first().alias("hour_most_popular"),
    pl.col("popularity_by_hour").first().alias("popularity_by_hour")
])


lds_popularity = hour_most_popular_church.filter(pl.col('location_name').str.contains('Church of Jesus Christ of'))

non_lds_popularity = hour_most_popular_church.filter(~pl.col('location_name').str.contains('Church of Jesus Christ of'))


In [ ]:
ggplot(lds_popularity, aes(x='hour_most_popular', y='region', fill='popularity_by_hour')) +\
           geom_violin() +\
           scale_fill_gradient(low="lightblue", high="darkblue") +\
           labs(title='Popularity by Region and Hour', x='Hour of Day', y='Region', fill='Popularity') +\
           theme_minimal()

In [ ]:
ggplot(non_lds_popularity, aes(x='hour_most_popular', y='region', fill='popularity_by_hour')) +\
           geom_violin() +\
           scale_fill_gradient(low="lightblue", high="darkblue") +\
           labs(title='Popularity by Region and Hour', x='Hour of Day', y='Region', fill='Popularity') +\
           theme_minimal()

##Contrast the `related_same_day_brand` brands between those who visit the Church of Jesus Christ of Latter-day Saints and those who visit other churches.

In [ ]:
Q3 = mergedata.select("placekey", "location_name", "related_same_day_brand", "region").explode("related_same_day_brand").unnest("related_same_day_brand")


Q3 =Q3.with_columns(
    max_correlation_church = pl.col("value").max().over("placekey"),
    brand_most_correllated = pl.when(pl.col("value") == pl.col("value").max().over("placekey"))\
        .then(pl.col("key")).otherwise(None).alias("brand_most_correllated")
)

most_correllated = brands.filter(~pl.all_horizontal(pl.col("brand_most_correllated").is_null()))


lds_related = mergedata.filter(pl.col('location_name').str.contains('Church of Jesus Christ of')) \
    .select('related_same_day_brand').explode('related_same_day_brand') \
    .group_by('related_same_day_brand').agg(pl.count().alias('count'))

non_lds_related = mergedata.filter(~pl.col('location_name').str.contains('Church of Jesus Christ of')) \
    .select('related_same_day_brand').explode('related_same_day_brand') \
    .group_by('related_same_day_brand').agg(pl.count().alias('count'))


print("Top related brands for LDS churches:")
print(lds_related.sort('count', descending=True).limit(10))

print("\nTop related brands for non-LDS churches:")
print(non_lds_related.sort('count', descending=True).limit(10))



<ipython-input-35-210c208e965a>:15: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .group_by('related_same_day_brand').agg(pl.count().alias('count'))
<ipython-input-35-210c208e965a>:19: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .group_by('related_same_day_brand').agg(pl.count().alias('count'))


Top related brands for LDS churches:
shape: (10, 2)
┌─────────────────────────────────┬───────┐
│ related_same_day_brand          ┆ count │
│ ---                             ┆ ---   │
│ struct[2]                       ┆ u32   │
╞═════════════════════════════════╪═══════╡
│ {"Chevron",1}                   ┆ 982   │
│ {"Target",1}                    ┆ 883   │
│ {"Chevron",2}                   ┆ 853   │
│ {"McDonald's",2}                ┆ 810   │
│ {"Costco",2}                    ┆ 804   │
│ {"Maverik",2}                   ┆ 711   │
│ {"The Home Depot",1}            ┆ 703   │
│ {"Wendy's",1}                   ┆ 697   │
│ {"Vasa Fitness",1}              ┆ 696   │
│ {"Smith's Food & Drug Stores",… ┆ 664   │
└─────────────────────────────────┴───────┘

Top related brands for non-LDS churches:
shape: (10, 2)
┌────────────────────────┬───────┐
│ related_same_day_brand ┆ count │
│ ---                    ┆ ---   │
│ struct[2]              ┆ u32   │
╞════════════════════════╪═══════╡
│ null      

In [ ]:
ggplot(lds_related.sort('count', descending=True).head(15)) + \
    aes(x='related_same_day_brand', y='count') + \
    geom_violin() + \
    labs(title='Distribution of Counts by Related Brand', x='Brand', y='Count') + \
    theme_minimal()

In [ ]:
ggplot(non_lds_related.sort('count', descending=True).head(15)) + \
    aes(x='related_same_day_brand', y='count') + \
    geom_violin() + \
    labs(title='Distribution of Counts by Related Brand', x='Brand', y='Count') + \
    theme_minimal()

##This is a bit of a reach. Compare `related_same_day_brand` of temples, seminary buildings, and meetinghouses of The Church of Jesus Christ of Latter-day Saints.

In [ ]:

lds_locations = mergedata.filter(pl.col('location_name').str.contains('Church of Jesus Christ of'))

temple_keywords = ['Temple']
seminary_keywords = ['Seminary']
meetinghouse_keywords = ['Meetinghouse']

def filter_locations(df, keywords):
  return df.filter(pl.any_horizontal([pl.col('location_name').str.contains(keyword) for keyword in keywords]))

temples = filter_locations(lds_locations, temple_keywords)
seminaries = filter_locations(lds_locations, seminary_keywords)
meetinghouses = filter_locations(lds_locations, meetinghouse_keywords)


def analyze_related_brands(df):
  related_brands = df.select('related_same_day_brand').explode('related_same_day_brand') \
      .group_by('related_same_day_brand').agg(pl.count().alias('count'))
  return related_brands.sort('count', descending=True).limit(10)


temple_related_brands = analyze_related_brands(temples)
seminary_related_brands = analyze_related_brands(seminaries)
meetinghouse_related_brands = analyze_related_brands(meetinghouses)


print("Top related brands for LDS temples:")
print(temple_related_brands)

print("\nTop related brands for LDS seminaries:")
print(seminary_related_brands)

print("\nTop related brands for LDS meetinghouses:")
print(meetinghouse_related_brands)

Top related brands for LDS temples:
shape: (10, 2)
┌─────────────────────────────────┬───────┐
│ related_same_day_brand          ┆ count │
│ ---                             ┆ ---   │
│ struct[2]                       ┆ u32   │
╞═════════════════════════════════╪═══════╡
│ {"Costco Gasoline",4}           ┆ 2     │
│ {"Costco",4}                    ┆ 2     │
│ {"Walmart",6}                   ┆ 2     │
│ {"Subway",2}                    ┆ 1     │
│ {"Maverik",2}                   ┆ 1     │
│ {"Olive Garden",1}              ┆ 1     │
│ {"Smith's Food & Drug Stores",… ┆ 1     │
│ {"Clarion",2}                   ┆ 1     │
│ {"In-N-Out Burger",1}           ┆ 1     │
│ {"Vasa Fitness",3}              ┆ 1     │
└─────────────────────────────────┴───────┘

Top related brands for LDS seminaries:
shape: (0, 2)
┌────────────────────────┬───────┐
│ related_same_day_brand ┆ count │
│ ---                    ┆ ---   │
│ struct[2]              ┆ u32   │
╞════════════════════════╪═══════╡
└───────────────

<ipython-input-45-17f89edc9447>:17: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .group_by('related_same_day_brand').agg(pl.count().alias('count'))


In [ ]:
from google.colab import files
f = files.upload()

# Convert ipynb to html
import subprocess
file0 = list(f.keys())[0]
_ = subprocess.run(["pip", "install", "nbconvert"])
_ = subprocess.run(["jupyter", "nbconvert", file0, "--to", "html"])

# download the html
files.download(file0[:-5]+"html")

Saving church_challenge.ipynb to church_challenge.ipynb


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/git@github.com:Reigns-Liu/Colab-notbooks.git
!git add .
!git commit -m "更新 notebook"
!git push

[Errno 2] No such file or directory: '/content/git@github.com:Reigns-Liu/Colab-notbooks.git'
/content
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
